<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# The Characteristic GRFFE speeds
## Author: Patrick Nelson

This notebook documents the function from the original `GiRaFFE` that calculates the speeds of the characteristic GRFFE hydrodynamics waves.

**Notebook Status:** <font color=green><b> Validated </b></font>

**Validation Notes:** This code has been validated to round-off level agreement with the corresponding code in the original `GiRaFFE` by means of its implementation in [Tutorial-GiRaFFE_NRPy-Stilde-flux](Tutorial-GiRaFFE_NRPy-Stilde-flux.ipynb) and [Tutorial-GiRaFFE_NRPy-Afield_flux_handwritten](Tutorial-GiRaFFE_NRPy-Afield_flux_handwritten.ipynb)

### NRPy+ Source Code for this module: 
* [GiRaFFE_NRPy/GiRaFFE_NRPy_Characteristic_Speeds.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_Characteristic_Speeds.py)

## Introduction

Here, we will find the speeds at which the hydrodynamics waves propagate. We start from the speed of light (since FFE deals with very diffuse plasmas), which is $c=1.0$ in our chosen units. We then find the speeds $c_+$ and $c_-$ on each face with the function `find_cp_cm`; then, we find minimum and maximum speeds possible from among those. A complete derivation of these speeds is included [below](#derive_speed) as an appendix; here, for brevity, we simply start with the implementation of the code in the original `GiRaFFE`. That code itself is adapted from the more general `IllinoisGRMHD` code; we will implement several more simplifications to help improve performance.

Below is the source code for `find_cp_cm`, edited to work with the NRPy+ version of `GiRaFFE`. One edit we need to make in particular is to the term `psim4*gupii` in the definition of `c`; that was written assuming the use of the conformal metric $\tilde{g}^{ii}$. Since we are not using that here, and are instead using the ADM metric, we should not multiply by $\psi^{-4}$.

```c
static inline void find_cp_cm(REAL &cplus,REAL &cminus,const REAL v02,const REAL u0,
                              const REAL vi,const REAL lapse,const REAL shifti,
                              const REAL gammadet,const REAL gupii) {
  const REAL u0_SQUARED=u0*u0;
  const REAL ONE_OVER_LAPSE_SQUARED = 1.0/(lapse*lapse);
  // sqrtgamma = psi6 -> psim4 = gammadet^(-1.0/3.0)
  const REAL psim4 = pow(gammadet,-1.0/3.0);
  //Find cplus, cminus:
  const REAL a = u0_SQUARED * (1.0-v02) + v02*ONE_OVER_LAPSE_SQUARED;
  const REAL b = 2.0* ( shifti*ONE_OVER_LAPSE_SQUARED * v02 - u0_SQUARED * vi * (1.0-v02) );
  const REAL c = u0_SQUARED*vi*vi * (1.0-v02) - v02 * ( gupii -
                                                               shifti*shifti*ONE_OVER_LAPSE_SQUARED);
  REAL detm = b*b - 4.0*a*c;
  //ORIGINAL LINE OF CODE:
  //if(detm < 0.0) detm = 0.0;
  //New line of code (without the if() statement) has the same effect:
  detm = sqrt(0.5*(detm + fabs(detm))); /* Based on very nice suggestion from Roland Haas */
  
  cplus = 0.5*(detm-b)/a;
  cminus = -0.5*(detm+b)/a;
  if (cplus < cminus) {
    const REAL cp = cminus;
    cminus = cplus;
    cplus = cp;
  }
}
```
Comments documenting this have been excised for brevity, but are reproduced in $\LaTeX$ [below](#derive_speed).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#prelim): Preliminaries
1. [Step 2](#simplifications): Apply GRFFE assumptions to simplify expressions
1. [Step 3](#char_speeds): Compute the characteristic speeds
    1. [Step 3.a](#cp_cm): Compute $c_+$ and $c_-$
    1. [Step 3.b](#cmax_cmin): Compute $c_\max$ and $c_\min$
1. [Step 4](#code_validation): Code Validation against `GiRaFFE_NRPy.GiRaFFE_NRPy_Characteristic_Speeds` NRPy+ Module
1. [Step 5](#derive_speed): Complete Derivation of the Wave Speeds
1. [Step 6](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file



<a id='prelim'></a>

# Step 1: Preliminaries \[Back to [top](#toc)\]
$$\label{prelim}$$

This first block of code imports the core NRPy+ functionality after first adding the main NRPy+ directory to the path. 

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
GRMHD_dir_path = os.path.join("../GRMHD_formulation/")
sys.path.append(GRMHD_dir_path)
import GRMHD_equations_new_version as GRMHD    # NRPy+: Generate general relativistic magnetohydrodynamics equations

nrpy_dir_path = os.path.join("../nrpy/nrpytutorial/")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

from outputC import outputC, outCfunction, outC_function_dict # NRPy+: Core C code output module
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

thismodule = "IGM-fluxes"

Ccodesdir = "IGM_standalone_Ccodes/"
cmd.mkdir(os.path.join(Ccodesdir))

#Step 0: Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

CoordSystem = "Cartesian"

# Set coordinate system to dst_basis
par.set_parval_from_str("reference_metric::CoordSystem", CoordSystem)
rfm.reference_metric()

<a id='simplifications'></a>

# Step 2: Apply GRFFE assumptions to simplify expressions \[Back to [top](#toc)\]
$$\label{simplifications}$$

We could use this code directly, but there's substantial improvement we can make by changing the code into a NRPyfied form. Note the `if` statement; NRPy+ does not know how to handle these, so we must eliminate it if we want to leverage NRPy+'s full power. (Calls to `fabs()` are also cheaper than `if` statements.) This can be done if we rewrite this, taking inspiration from the other eliminated `if` statement documented in the above code block:
```c
  cp = 0.5*(detm-b)/a;
  cm = -0.5*(detm+b)/a;
  cplus  = 0.5*(cp+cm+fabs(cp-cm));
  cminus = 0.5*(cp+cm-fabs(cp-cm));
```
This can be simplified further, by substituting `cp` and `cm` into the below equations and eliminating terms as appropriate. First note that `cp+cm = -b/a` and that `cp-cm = detm/a`. Thus,
```c
  cplus  = 0.5*(-b/a + fabs(detm/a));
  cminus = 0.5*(-b/a - fabs(detm/a));
```
This fulfills the original purpose of the `if` statement in the original code because we have guaranteed that $c_+ \geq c_-$.

This leaves us with an expression that can be much more easily NRPyfied. So, we will rewrite the following in NRPy+, making only minimal changes to be proper Python. However, it turns out that we can make this even simpler. In GRFFE, $v_0^2$ is guaranteed to be exactly one. In GRMHD, this speed was calculated as $$v_{0}^{2} = v_{\rm A}^{2} + c_{\rm s}^{2}\left(1-v_{\rm A}^{2}\right),$$ where the Alfv&eacute;n speed $v_{\rm A}^{2}$ $$v_{\rm A}^{2} = \frac{b^{2}}{\rho_{b}h + b^{2}}.$$ So, we can see that when the density $\rho_b$ goes to zero, $v_{0}^{2} = v_{\rm A}^{2} = 1$. Then 
\begin{align}
a &= (u^0)^2 (1-v_0^2) + v_0^2/\alpha^2 \\
&= 1/\alpha^2 \\
b &= 2 \left(\beta^i v_0^2 / \alpha^2 - (u^0)^2 v^i (1-v_0^2)\right) \\
&= 2 \beta^i / \alpha^2 \\
c &= (u^0)^2 (v^i)^2 (1-v_0^2) - v_0^2 \left(\gamma^{ii} - (\beta^i)^2/\alpha^2\right) \\
&= -\gamma^{ii} + (\beta^i)^2/\alpha^2,
\end{align}
are simplifications that should save us some time; we can see that $a \geq 0$ is guaranteed. Note that we also force `detm` to be positive. Thus, `detm/a` is guaranteed to be positive itself, rendering the calls to `nrpyAbs()` superfluous. Furthermore, we eliminate any dependence on the Valencia 3-velocity and the time compoenent of the four-velocity, $u^0$. This leaves us free to solve the quadratic in the familiar way: $$c_\pm = \frac{-b \pm \sqrt{b^2-4ac}}{2a}$$.

<a id='find_cp_cm'></a>

# Step 2: The `find_cp_cm()` function \[Back to [top](#toc)\]
$$\label{find_cp_cm}$$

We will now explain the inlined function `find_cp_cm`. Keep in mind that this function depend on the function `compute_v02`, [which is implemented below](#compute_v02). This function is called with the objective of computing the minimum ($-$) and maximum ($+$) characteristic speeds at each cell interface, $c_{\pm}^{r,l}$.

We approximate the general GRMHD dispersion relation (eq. 27 of [Gammie & McKinney (2003)](https://arxiv.org/pdf/astro-ph/0301509.pdf)) by the simpler expression

$$
\omega_{\rm cm}^{2} = \left[v_{\rm A}^{2} + c_{\rm s}^{2}\left(1-v_{\rm A}^{2}\right)\right]k_{\rm cm}^{2}\ ,
$$

where $\omega_{\rm cm}=-k_{\mu}u^{\mu}$ is the frequency and $k_{\rm cm}^{2} = K_{\mu}K^{\mu}$ the wavenumber of an MHD wave mode in the frame comoving with the fluid, where $K_{\mu}$ is defined as the projection of the wave vector $k^{\nu}$ onto the direction normal to $u^{\nu}$: $K_{\mu} = \left(g_{\mu\nu}+u_{\mu}u_{\nu}\right)k^{\nu}$. $c_{\rm s}$ is the sound speed, and $v_{\rm A}$ is the Alfvén speed, given by

$$
v_{\rm A} = \sqrt{\frac{b^{2}}{\rho_{b}h + b^{2}}}\ .
$$

With these definitions, we may then solve the approximate dispersion relation above along direction $i$, noting that in the comoving frame $k_{\mu} = \left(-\omega,k_{j}\delta^{j}_{\ i}\right)$ and the wave (phase) velocity is $c_{\pm} = \left.\omega\middle/\left(k_{j}\delta^{j}_{\ i}\right)\right.$. The dispersion can then be written as a quadratic equation for $c_{\pm}$:

$$
ac_{\pm}^{2} + bc_{\pm} + c = 0\ ,
$$

with

$$
\boxed{
\begin{align}
a &= \left(1-v_{0}^{2}\right)\left(u^{0}\right)^{2} - v_{0}^{2}g^{00}\ ,\\
b &= 2v_{0}^{2}g^{i0} - 2u^{i}u^{0}\left(1-v^{2}_{0}\right)\ ,\\
c &= \left(1-v_{0}^{2}\right)\left(u^{i}\right)^{2} - v_{0}^{2}g^{ii}\ ,\\
v_{0}^{2} &= v_{\rm A}^{2} + c_{\rm s}^{2}\left(1-v_{\rm A}^{2}\right)\ ,\\
c_{\rm s} &= \left.\left[\frac{dP_{\rm cold}}{d\rho_{b}} + \Gamma_{\rm th}\left(\Gamma_{\rm th}-1\right)\epsilon_{\rm th}\right]\middle/h\right.\ ,\\
c_{+} &= \max\left(\frac{-b \pm \sqrt{b^{2}-4ac}}{2a}\right)\ ,\\
c_{-} &= \min\left(\frac{-b \pm \sqrt{b^{2}-4ac}}{2a}\right)\ .
\end{align}
}
$$

For the implementation of $v_{0}^{2}$, please see [Step 4 below](#compute_v02).

In [2]:
# We'll write this as a function so that we can calculate the expressions on-demand for any choice of i
def find_cp_cm(flux_dirn, g4UU, 
               smallbsquared, u4U, Gamma_th, 
               epsilon_th, dPcold_drhob, h, rho_b):
    # Outputs: cplus,cminus
    vA = sp.sqrt( smallbsquared / (rho_b*h + smallbsquared) )
    cs = (dPcold_drhob + Gamma_th*(Gamma_th-1)*epsilon_th) / h
    v02 = vA**2 + (cs**2)*(1 - vA**2)
    
    a = (1 - v02)*(u4U[0]**2) - v02*g4UU[0][0]
    b = 2*v02*g4UU[flux_dirn+1][0] - 2*u4U[flux_dirn+1]*u4U[0]*(1 - v02)
    c = (1 - v02)*(u4U[flux_dirn+1]**2) - v02*g4UU[flux_dirn+1][flux_dirn+1]

    # Now, we are free to solve the quadratic equation as usual. We take care to avoid passing a
    # negative value to the sqrt function.
    detm = b*b - sp.sympify(4)*a*c

    import Min_Max_and_Piecewise_Expressions as noif
    detm = sp.sqrt(noif.max_noif(sp.sympify(0),detm))
    global cplus,cminus
    # note that these correspond to a single interface, left or right
    cplus_tmp  = sp.Rational(1,2)*(-b/a + detm/a)
    cminus_tmp = sp.Rational(1,2)*(-b/a - detm/a)
    
    cminus =   noif.coord_less_bound(cplus_tmp, cminus_tmp)*cplus_tmp\
             + noif.coord_greater_bound(cplus_tmp, cminus_tmp)*cminus_tmp

    cplus =   noif.coord_less_bound(cplus_tmp, cminus_tmp)*cminus_tmp\
            + noif.coord_greater_bound(cplus_tmp, cminus_tmp)*cplus_tmp

    # the above in C code
    # if (cplus < cminus) {
    # CCTK_REAL cp = cminus;
    # cminus = cplus;
    # cplus = cp;

<a id='cmax_cmin'></a>

## Step 3.b: Compute $c_\max$ and $c_\min$ \[Back to [top](#toc)\]
$$\label{cmax_cmin}$$

In flat spacetime, where $\alpha=1$, $\beta^i=0$, and $\gamma^{ij} = \delta^{ij}$, $c_+ > 0$ and $c_- < 0$. For the HLLE solver, we will need both `cmax` and `cmin` to be positive; we also want to choose the speed that is larger in magnitude because overestimating the characteristic speeds will help damp unwanted oscillations. (However, in GRFFE, we only get one $c_+$ and one $c_-$, so we only need to fix the signs here.) Hence, the following function.  

We will now write a function in NRPy+ similar to the one used in the old `GiRaFFE`, allowing us to generate the expressions with less need to copy-and-paste code; the key difference is that this one will be in Python, and generate optimized C code integrated into the rest of the operations. Notice that since we eliminated the dependence on velocities, none of the input quantities are different on either side of the face. So, this function won't really do much besides guarantee that `cmax` and `cmin` are positive, but we'll leave the machinery here since it is likely to be a useful guide to somebody who wants to something similar. We use the same technique as above to replace the `if` statements inherent to the `MAX()` and `MIN()` functions.

In [3]:
# We'll write this as a function, and call it within HLLE_solver, below.
def find_cmax_cmin(flux_dirn, gamma_faceDD, beta_faceU, alpha_face,
                   smallbsquared_r, smallbsquared_l, u4U_r, u4U_l, 
                   Gamma_th_r, Gamma_th_l, 
                   epsilon_th_r, epsilon_th_l, dPcold_drhob_r, dPcold_drhob_l, 
                   h_r, h_l, rho_b_r, rho_b_l):
    # Inputs:  flux direction flux_dirn, Inverse metric g4_faceUU, shift beta_faceU,
    #          lapse alpha_face, metric determinant gammadet_face
    # Outputs: maximum and minimum characteristic speeds cmax and cmin
    # First, we need to find the characteristic speeds on each face
    import BSSN.ADMBSSN_tofrom_4metric as AB4m
    AB4m.g4UU_ito_BSSN_or_ADM("ADM",gamma_faceDD, beta_faceU, alpha_face)

    # Original needed for GRMHD
    find_cp_cm(flux_dirn, AB4m.g4UU, smallbsquared_r, u4U_r, Gamma_th_r, epsilon_th_r, dPcold_drhob_r, h_r, rho_b_r)
    cpr = cplus
    cmr = cminus
    
    find_cp_cm(flux_dirn, AB4m.g4UU, smallbsquared_l, u4U_l, Gamma_th_l, epsilon_th_l, dPcold_drhob_l, h_l, rho_b_l)
    cpl = cplus
    cml = cminus

    # The following algorithms have been verified with random floats:

    global cmax,cmin    
#   // Then compute cmax, cmin. This is required for the HLL flux.
#   original C code
#   CCTK_REAL cmaxL =  MAX(0.0,MAX(cplusl,cplusr));
#   CCTK_REAL cminL = -MIN(0.0,MIN(cminusl,cminusr));
    
    # Now, we need to set cmax to the larger of cpr,cpl, and 0
    import Min_Max_and_Piecewise_Expressions as noif
    cmax = noif.max_noif(0.0, noif.max_noif(cpl, cpr))
    # And then, set cmin to the smaller of cmr,cml, and 0
    cmin = -noif.min_noif(0.0, noif.min_noif(cml, cmr))

<a id='code_validation'></a>

# Step 4:  Code Validation against `GiRaFFE_NRPy.GiRaFFE_NRPy_Characteristic_Speeds` NRPy+ Module \[Back to [top](#toc)\]
$$\label{code_validation}$$


Here, as a code validation check, we verify agreement in the SymPy expressions for the `GiRaFFE` evolution equations and auxiliary quantities we intend to use between
1. this tutorial and 
2. the NRPy+ [GiRaFFE_NRPy.GiRaFFE_NRPy_Characteristic_Speeds](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_Characteristic_Speeds.py) module.

This first validation directly compares the sympy expressions. This is generally quicker and more reliable, but might overlook some complexities in implementing the C code.

In [4]:
formalism="BSSN"

sqrt4pi = sp.symbols("sqrt4pi")
alpha_face = sp.symbols("alpha_face")
if formalism=="BSSN":
    cf_face = sp.symbols("cf_face")
    h_faceDD = ixp.declarerank2("h_faceDD","sym01",DIM=3)
    vet_faceU = ixp.declarerank1("vet_faceU", DIM=3)
    beta_faceU = vet_faceU
    
    import BSSN.ADM_in_terms_of_BSSN as AitoB
    AitoB.ADM_in_terms_of_BSSN()

    import BSSN.BSSN_quantities as Bq

    gamma_faceDD = ixp.zerorank2()
    for i in range(3):
        for j in range(3):
            gamma_faceDD[i][j] = AitoB.gammaDD[i][j].subs(Bq.hDD[i][j], h_faceDD[i][j]).subs(Bq.cf, cf_face)

else:
    beta_faceU = ixp.declarerank1("beta_faceU") 
    gamma_faceDD = ixp.declarerank2("gamma_faceDD","sym01")
            
            
# We'll need some more gridfunctions, now, to represent the reconstructions of BU and ValenciavU
# on the right and left faces
u4_rU = ixp.declarerank1("u4_rU", DIM=4)
u4_lU = ixp.declarerank1("u4_lU", DIM=4)

B_rU = ixp.declarerank1("B_rU")
B_lU = ixp.declarerank1("B_lU")

cmins = ["conservative_fluxes->cmin_dirn0", 
         "conservative_fluxes->cmin_dirn1", 
         "conservative_fluxes->cmin_dirn2"]

cmaxs = ["conservative_fluxes->cmax_dirn0", 
         "conservative_fluxes->cmax_dirn1", 
         "conservative_fluxes->cmax_dirn2"]

h_r = sp.symbols("h_r")
h_l = sp.symbols("h_l")

rho_b_r = sp.symbols("rhob_r")
rho_b_l = sp.symbols("rhob_l")

Gamma_th_r = sp.symbols("Gamma_th_r")
Gamma_th_l = sp.symbols("Gamma_th_l")

epsilon_th_r = sp.symbols("epsilon_th_r")
epsilon_th_l = sp.symbols("epsilon_th_l")

dPcold_drhob_r = sp.symbols("dPcold_drhob_r")
dPcold_drhob_l = sp.symbols("dPcold_drhob_l")

GRMHD.compute_smallb4U(gamma_faceDD, beta_faceU, alpha_face, u4_rU, B_rU, sqrt4pi)
GRMHD.compute_smallbsquared(gamma_faceDD, beta_faceU, alpha_face, GRMHD.smallb4U)
smallbsquared_r = GRMHD.smallbsquared

GRMHD.compute_smallb4U(gamma_faceDD, beta_faceU, alpha_face, u4_lU, B_lU, sqrt4pi)
GRMHD.compute_smallbsquared(gamma_faceDD, beta_faceU, alpha_face, GRMHD.smallb4U)
smallbsquared_l = GRMHD.smallbsquared

In [5]:
cmins_rhs = []
cmaxs_rhs = []

for flux_dirn in range(3):
    find_cmax_cmin(flux_dirn, gamma_faceDD, beta_faceU, alpha_face,
                   smallbsquared_r, smallbsquared_l, u4_rU, u4_lU, 
                   Gamma_th_r, Gamma_th_l, 
                   epsilon_th_r, epsilon_th_l, dPcold_drhob_r, dPcold_drhob_l, 
                   h_r, h_l, rho_b_r, rho_b_l)
    
    cmins_rhs.append(cmin)
    cmaxs_rhs.append(cmax)

In [6]:
prims_velocities = ["u4_rU0", "u4_rU1", "u4_rU2", "u4_rU3",
                    "u4_lU0", "u4_lU1", "u4_lU2", "u4_lU3"]

prims_mag_field = ["B_rU0", "B_rU1", "B_rU2",
                   "B_lU0", "B_lU1", "B_lU2"]
    
other_prims = ["P", "h", "rhob","Gamma_th", "epsilon_th", "dPcold_drhob"]

params = ["GAMMA_SPEED_LIMIT", "TINYDOUBLE", "sqrt4pi"]

prestring = ""

for var in prims_velocities + prims_mag_field:
    prestring += "const double "+str(var)+" = reconstructed_prims->"+str(var)+";\n"

for var in other_prims:
    prestring += "const double "+str(var)+"_r = reconstructed_prims->"+str(var)+"_r;\n"
    prestring += "const double "+str(var)+"_l = reconstructed_prims->"+str(var)+"_l;\n"
for var in params:
    prestring += "const double "+str(var)+" = rhss_params->"+str(var)+";\n"

prestring += "const double "+str(alpha_face)+" = metric_face_quantities->"+str(alpha_face)+";\n"

if formalism=="BSSN":
    prestring += "const double "+str(cf_face)+" = metric_face_quantities->"+str(cf_face)+";\n"

    for i in range(3):
        vetU_var = vet_faceU[i]
        prestring += "const double "+str(vetU_var)+" = metric_face_quantities->"+str(vetU_var)+";\n"

    for i in range(3):
        for j in range(3):
            hDD_var = h_faceDD[i][j]
            prestring += "const double "+str(hDD_var)+" = metric_face_quantities->"+str(hDD_var)+";\n"

else:
    for i in range(3):
            betaU_var = beta_faceU[i]
            prestring += "const double "+str(betaU_var)+" = metric_face_quantities->"+str(betaU_var)+";\n"

    for i in range(3):
        for j in range(3):
            gammaDD_var = gamma_faceDD[i][j]
            prestring += "const double "+str(gammaDD_var)+" = metric_face_quantities->"+str(gammaDD_var)+";\n"

In [7]:
outCparams = "outCverbose=False,CSE_sorting=True"

c_type = "void"
params   = "const rhss_paramstruct *restrict rhss_params, "
params  += "const prims_struct *restrict reconstructed_prims, "
params  += "const metric_quantities_struct *restrict metric_face_quantities, "
params  += "conservative_fluxes_struct *restrict conservative_fluxes"

for flux_dirn in range(3):
    write_speeds_str = [cmins[flux_dirn], cmaxs[flux_dirn]]
    write_speeds_rhs_str = [cmins_rhs[flux_dirn], cmaxs_rhs[flux_dirn]]

    body = outputC(write_speeds_rhs_str, write_speeds_str, params=outCparams, 
                   filename="returnstring", prestring=prestring)

    desc = "Compute the characteristic speeds in" + str(flux_dirn) +"th direction"
    name = "calculate_characteristic_speed_" + str(flux_dirn) +"th_direction"

    outCfunction(
    outfile=os.path.join(Ccodesdir,name+".h"),
    #     includes=includes,
    desc=desc,
    name=name,
    params=params,
    body= body, 
    enableCparameters=False)

Output C function calculate_characteristic_speed_0th_direction() to file IGM_standalone_Ccodes/calculate_characteristic_speed_0th_direction.h
Output C function calculate_characteristic_speed_1th_direction() to file IGM_standalone_Ccodes/calculate_characteristic_speed_1th_direction.h
Output C function calculate_characteristic_speed_2th_direction() to file IGM_standalone_Ccodes/calculate_characteristic_speed_2th_direction.h


In [8]:
files = ["calculate_characteristic_speed_0th_direction.h",
         "calculate_characteristic_speed_1th_direction.h",
         "calculate_characteristic_speed_2th_direction.h"]

names = ["calculate_characteristic_speed_0th_direction",
         "calculate_characteristic_speed_1th_direction",
         "calculate_characteristic_speed_2th_direction"]

In [9]:
# Define the directory that we wish to validate against:
valdir = "IGM_Ccode_library/"
cmd.mkdir(os.path.join(valdir))

import IGM_Characteristic_Speeds as chsp
chsp.add_to_Cfunction_dict__GRMHD_characteristic_speeds(formalism=formalism,
                                      outCparams=outCparams)

for name in names:
    with open(os.path.join(valdir,name+".h"), "w") as file:
        file.write(outC_function_dict[name])

import difflib
import sys

print("Printing difference between original C code and this code...")
# Open the files to compare

for file in files:
    print("Checking file " + file)
    with open(os.path.join(valdir,file)) as file1, open(os.path.join(Ccodesdir,file)) as file2:
        # Read the lines of each file
        file1_lines = file1.readlines()
        file2_lines = file2.readlines()
        num_diffs = 0
        for line in difflib.unified_diff(file1_lines, file2_lines, fromfile=os.path.join(valdir+file), tofile=os.path.join(Ccodesdir+file)):
            sys.stdout.writelines(line)
            num_diffs = num_diffs + 1
        if num_diffs == 0:
            print("No difference. TEST PASSED!")
        else:
            print("ERROR: Disagreement found with .h file. See differences above.")
            sys.exit(1)

Printing difference between original C code and this code...
Checking file calculate_characteristic_speed_0th_direction.h
--- IGM_Ccode_library/calculate_characteristic_speed_0th_direction.h
+++ IGM_standalone_Ccodes/calculate_characteristic_speed_0th_direction.h
@@ -1,37 +1,38 @@
-#include "./NRPy_basic_defines.h"
-#include "./NRPy_function_prototypes.h"
 /*
  * Compute the characteristic speeds in0th direction
  */
-void calculate_characteristic_speed_0th_direction(const reconstructed_prims_struct *restrict reconstructed_prims_r, const reconstructed_prims_struct *restrict reconstructed_prims_l,const metric_face_quantities_struct *restrict metric_face_quantities, conservative_fluxes_struct *restrict conservative_fluxes) {
+void calculate_characteristic_speed_0th_direction(const rhss_paramstruct *restrict rhss_params, const prims_struct *restrict reconstructed_prims, const metric_quantities_struct *restrict metric_face_quantities, conservative_fluxes_struct *restrict conservative_fluxe

SystemExit: 1

/home/terrence/environments/ve3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3369: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


<a id='derive_speed'></a>

# Step 5: Complete Derivation of the Wave Speeds \[Back to [top](#toc)\]
$$\label{derive_speed}$$

This computes phase speeds in the direction given by `flux_dirn`. Note that we replace the full dispersion relation with a simpler one, which overestimates the maximum speeds by a factor of ~2. See full discussion around Eqs. 49 and 50 in [Duez, et al.](http://arxiv.org/pdf/astro-ph/0503420.pdf). In summary, we solve the dispersion relation (in, e.g., the $x$-direction) with a wave vector of $k_\mu = (-\omega,k_x,0,0)$. So, we solve the approximate dispersion relation $\omega_{\rm cm}^2 = [v_A^2 + c_s^2 (1-v_A^2)]k_{\rm cm}^2$ for the wave speed $\omega/k_x$, where the sound speed $c_s = \sqrt{\Gamma P/(h \rho_0)}$, the Alfv&eacute;n speed $v_A = 1$ (in GRFFE), $\omega_{\rm cm} = -k_\mu k^\mu$ is the frequency in the comoving frame, $k_{\rm cm}^2 = K_\mu K^\mu$ is the wavenumber squared in the comoving frame, and $K_\mu = (g_{\mu\nu} + u_\mu u_\nu)k^\nu$ is the part of the wave vector normal to the four-velocity $u^\mu$. See below for a complete derivation.

What follows is a complete derivation of the quadratic we solve. We start from the following relations:
\begin{align}
w_{\rm cm} &= (-k_0 u^0 - k_x u^x) \\
k_{\rm cm}^2 &= K_{\mu} K^{\mu}, \\
K_{\mu} K^{\mu} &= (g_{\mu a} + u_{\mu} u_a) k^a  g^{\mu b} [ (g_{c b} + u_c u_b) k^c ] \\
\end{align}
The last term of the above can be written as follow:
$$
(g_{c b} + u_{c} u_{b}) k^c = (\delta^{\mu}_c + u_c u^{\mu} ) k^c
$$

Then,
\begin{align}
K_{\mu} K^{\mu} &= (g_{\mu a} + u_{\mu} u_a) k^a  g^{\mu b} [ (g_{c b} + u_c u_b) k^c ] \\
                 &= (g_{\mu a} + u_{\mu} u_a) k^a  (\delta^{\mu}_c + u_c u^{\mu} ) k^c \\
                 &=[(g_{\mu a} + u_{\mu} u_a) \delta^{\mu}_c + (g_{\mu a} + u_{\mu} u_a) u_c u^{\mu} ] k^c k^a \\
                 &=[(g_{c a} + u_c u_a) + (u_c u_a -  u_a u_c] k^c k^a \\
                 &=(g_{c a} + u_c u_a) k^c k^a \\
                 &= k_a k^a + u^c u^a k_c k_a \\
k^a = g^{\mu a} k_{\mu} &= g^{0 a} k_0 + g^{x a} k_x \\
k_a k^a &= k_0 g^{0 0} k_0 + k_x k_0 g^{0 x} + g^{x 0} k_0 k_x + g^{x x} k_x k_x \\
         &= g^{00} (k_0)^2 + 2 g^{x0} k_0 k_x + g^{xx} (k_x)^2 \\
u^c u^a k_c k_a &= (u^0 k_0 + u^x k_x) (u^0 k_0 + u^x k_x) = (u^0 k_0)^2 + 2 u^x k_x u^0 k_0 + (u^x k_x)^2 \\
(k_0 u^0)^2  + 2 k_x u^x k_0 u^0 + (k_x u^x)^2 &= v_0^2 [ (u^0 k_0)^2 + 2 u^x k_x u^0 k_0 + (u^x k_x)^2 + g^{00} (k_0)^2 + 2 g^{x0} k_0 k_x + g^{xx} (k_x)^2] \\
(1-v_0^2) (u^0 k_0 + u^x k_x)^2 &= v_0^2 (g^{00} (k_0)^2 + 2 g^{x0} k_0 k_x + g^{xx} (k_x)^2) \\
(1-v_0^2) (u^0 k_0/k_x + u^x)^2 &= v_0^2 (g^{00} (k_0/k_x)^2 + 2 g^{x0} k_0/k_x + g^{xx}) \\
(1-v_0^2) (u^0 X + u^x)^2 &= v_0^2 (g^{00} X^2 + 2 g^{x0} X + g^{xx}) \\
(1-v_0^2) ((u^0)^2 X^2 + 2 u^x (u^0) X + (u^x)^2) &= v_0^2 (g^{00} X^2 + 2 g^{x0} X + g^{xx}) \\
0 &= X^2 ( (1-v_0^2) (u^0)^2 - v_0^2 g^{00}) + X (2 u^x u^0 (1-v_0^2) - 2 v_0^2 g^{x0}) + (1-v_0^2) (u^x)^2 - v_0^2 g^{xx} \\
a &= (1-v_0^2) (u^0)^2 - v_0^2 g^{00} = (1-v_0^2) (u^0)^2 + v_0^2/\alpha^2 \leftarrow {\rm VERIFIED} \\
b &= 2 u^x u^0 (1-v_0^2) - 2 v_0^2 \beta^x/\alpha^2 \leftarrow {\rm VERIFIED,\ } X\rightarrow -X, {\rm because\ } X = -w/k_1, {\rm \ and\ we\ are\ solving\ for} -X. \\
c &= (1-v_0^2) (u^x)^2 - v_0^2 (\gamma^{xx}\psi^{-4} - (\beta^x/\alpha)^2) \leftarrow {\rm VERIFIED} \\
v_0^2 &= v_A^2 + c_s^2 (1 - v_A^2) \\
\end{align}

<a id='latex_pdf_output'></a>

# Step 6: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFE_NRPy-Characteristic_Speeds.pdf](Tutorial-GiRaFFE_NRPy-Characteristic_Speeds.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [ ]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GiRaFFE_NRPy-Characteristic_Speeds",location_of_template_file=os.path.join(".."))